In [1]:
import json
from datetime import datetime
from pathlib import Path
import os

from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"

In [ ]:
# argument parser
# folder = Path('/dpt/veos/eos_jiangyin/HMZABAAHXNF005658−changlong-li/py_logs')
# folder = Path('/dpt/veos/eos_jiangyin/HMZABAAH1MF011055−wangcheng/py_logs')
# folder = Path('/dpt/veos/eos_jiangyin/HMZABAAH8NF005657−hongmei-chen/py_logs')
folder = Path('/dpt/veos/eos_jiangyin/HMZABAAHXMF011054−wangcheng/py_logs')

In [ ]:
# files = glob.glob(str(folder) + "/*.log")
files = folder.glob("*.log")
try:
    os.mkdir(folder / "converted")
except FileExistsError:
    print('Folder already exists')


for file in files:
    converted_target_file = (
        (file.resolve().parent / "converted" / file.name)
        .with_stem(f'{file.stem}_from_log')
        .with_suffix(".json")
    )
    # print(converted_target_file)

    with open(file) as f:
        # line by line
        lines = f.readlines()
        json_lines = []
        for l in lines:
            # remove \n
            l = l.strip("\n")
            # string to dict
            l = json.loads(l)
            created = datetime.fromisoformat(l["asctime"].replace(",", "."))
            l["created"] = created.timestamp()
            # append dict
            json_lines.append(l)
    with open(converted_target_file, 'w') as f:
        f.write(json.dumps(json_lines))
    print(f'Converted {file} to {converted_target_file}')
    # jsonFileName = file.split(".")
    # print(jsonFileName)

# Analyze error and warnings in the json files

In [2]:
import pandas as pd


In [51]:
# def get_tuple_from_name_lineno(x):
#     name = x["name"]
#     lineno = x["lineno"]
#     x['report_location'] = x['name'] + '_' + x['lineno'].astype(str)
#     return x



In [22]:
# files = glob.glob(str(folder) + "/*.log")
folders = [
    Path('/dpt/veos/eos_jiangyin/HMZABAAHXMF011054−wangcheng/py_logs'),
    Path('/dpt/veos/eos_jiangyin/HMZABAAHXNF005658−changlong-li/py_logs'),
    Path('/dpt/veos/eos_jiangyin/HMZABAAH1MF011055−wangcheng/py_logs'),
    Path('/dpt/veos/eos_jiangyin/HMZABAAH8NF005657−hongmei-chen/py_logs'),
]
report_loc_set = []
# folder = Path('/dpt/veos/eos_jiangyin/HMZABAAHXMF011054−wangcheng/py_logs')
# folder = Path('/dpt/veos/eos_jiangyin/HMZABAAHXNF005658−changlong-li/py_logs')
# folder = Path('/dpt/veos/eos_jiangyin/HMZABAAH1MF011055−wangcheng/py_logs')
# folder = Path('/dpt/veos/eos_jiangyin/HMZABAAH8NF005657−hongmei-chen/py_logs')
for folder in folders:
    files = folder.glob("*.json")
    try:
        os.mkdir(folder / "debug")
    except FileExistsError:
        # print('Folder already exists')
        pass

    for file in files:
        converted_target_file = (
            (file.resolve().parent / "debug" / file.name)
            .with_stem(f'{file.stem}_describe')
            .with_suffix(".json")
        )
        # print(converted_target_file)

        with open(file) as f:
            # line by line
            try:
                logs = pd.read_json(f, lines=True)
            except ValueError as e:
                print(f'Error in {file}: {e}')

        failures = logs[logs['levelname'].isin(['ERROR', 'WARNING'])]
        if len(failures) == 0:
            continue
        failures.set_index('asctime', inplace=True)
        # failures.to_json(converted_target_file)
        # failures['message'].values
        # failures[['name', 'levelname', 'lineno', 'message']].to_csv(
        #     converted_target_file.with_suffix('.csv')
        # )
        failures_loc = (
            failures.loc[:, 'module'] + '_' + failures.loc[:, 'lineno'].astype(str)
        )
        # failure_location = failures.groupby('name').apply(get_tuple_from_name_lineno)
        report_loc_set.append(set(failures_loc.unique().tolist()))
        print(len(report_loc_set))
        # if len(report_loc_set) % 10 == 0:
        #     print(report_loc_set)

        # failures = logs[
        #     (logs['levelname'] == 'ERROR') | (logs['levelname'] == 'WARNING'), ['message']
        # ]
        # failures.to_json(converted_target_file)
        # logs.shape

        # with open(converted_target_file, 'w') as f:
        #     f.write(json.dumps(json_lines))
        # print(f'Analysis of {file} in {converted_target_file}')
print(f'Found {len(report_loc_set)} unique report locations')
report_loc_set


/tmp/ipykernel_1556145/2954595795.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  failures.loc[:, 'report_location'] = (
/tmp/ipykernel_1556145/2954595795.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  failures.loc[:, 'report_location'] = (
/tmp/ipykernel_1556145/2954595795.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas

1
2
3


/tmp/ipykernel_1556145/2954595795.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  failures.loc[:, 'report_location'] = (


4


/tmp/ipykernel_1556145/2954595795.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  failures.loc[:, 'report_location'] = (
/tmp/ipykernel_1556145/2954595795.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  failures.loc[:, 'report_location'] = (


...

200
201
202


/tmp/ipykernel_1556145/2954595795.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  failures.loc[:, 'report_location'] = (


203
204
Found 204 unique report locations


/tmp/ipykernel_1556145/2954595795.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  failures.loc[:, 'report_location'] = (


[{'realtime_train_infer_ddpg_1157',
  'realtime_train_infer_ddpg_1374',
  'realtime_train_infer_ddpg_1379',
  'remote_can_client_304',
  'remote_can_client_332'},
 {'realtime_train_infer_ddpg_1157',
  'realtime_train_infer_ddpg_1374',
  'realtime_train_infer_ddpg_1379',
  'remote_can_client_304',
  'remote_can_client_332'},
 {'realtime_train_infer_ddpg_2009', 'remote_can_client_228'},
 {'realtime_train_infer_ddpg_1164',
  'realtime_train_infer_ddpg_1387',
  'realtime_train_infer_ddpg_2011',
  'remote_can_client_333'},
 {'realtime_train_infer_ddpg_1164',
  'realtime_train_infer_ddpg_1387',
  'remote_can_client_333'},
 {'realtime_train_infer_ddpg_1164',
  'realtime_train_infer_ddpg_1387',
  'remote_can_client_230',
  'remote_can_client_305'},
 {'realtime_train_infer_ddpg_1164',
  'realtime_train_infer_ddpg_1387',
  'remote_can_client_230',
  'remote_can_client_305',
  'remote_can_client_333'},
 {'realtime_train_infer_ddpg_1164',
  'realtime_train_infer_ddpg_1387',
  'realtime_train_infer

In [25]:
import csv

folder = Path('/dpt/veos/eos_jiangyin/report_locations.csv')
with open(folder, 'w') as f:
    writer = csv.writer(f)
    writer.writerow(report_loc_set)


23662

In [23]:
report_loc_set[0]
report_loc_set[100]
report_loc_set[0] | report_loc_set[100]

{'realtime_train_infer_ddpg_1157',
 'realtime_train_infer_ddpg_1374',
 'realtime_train_infer_ddpg_1379',
 'remote_can_client_304',
 'remote_can_client_332'}

{'realtime_train_infer_ddpg_1148',
 'realtime_train_infer_ddpg_1370',
 'realtime_train_infer_ddpg_2000',
 'remote_can_client_230',
 'remote_can_client_305',
 'remote_can_client_333'}

{'realtime_train_infer_ddpg_1148',
 'realtime_train_infer_ddpg_1157',
 'realtime_train_infer_ddpg_1370',
 'realtime_train_infer_ddpg_1374',
 'realtime_train_infer_ddpg_1379',
 'realtime_train_infer_ddpg_2000',
 'remote_can_client_230',
 'remote_can_client_304',
 'remote_can_client_305',
 'remote_can_client_332',
 'remote_can_client_333'}

In [26]:
all_locations = set()
for location in report_loc_set:
    all_locations |= location

all_locations
with open(folder, 'w') as f:
    writer = csv.writer(f)
    writer.writerow(all_locations)
    writer.writerow(report_loc_set)


{'realtime_train_infer_ddpg_1135',
 'realtime_train_infer_ddpg_1138',
 'realtime_train_infer_ddpg_1148',
 'realtime_train_infer_ddpg_1157',
 'realtime_train_infer_ddpg_1164',
 'realtime_train_infer_ddpg_1349',
 'realtime_train_infer_ddpg_1354',
 'realtime_train_infer_ddpg_1359',
 'realtime_train_infer_ddpg_1365',
 'realtime_train_infer_ddpg_1370',
 'realtime_train_infer_ddpg_1374',
 'realtime_train_infer_ddpg_1379',
 'realtime_train_infer_ddpg_1387',
 'realtime_train_infer_ddpg_2000',
 'realtime_train_infer_ddpg_2009',
 'realtime_train_infer_ddpg_2011',
 'realtime_train_infer_ddpg_2475',
 'realtime_train_infer_ddpg_2578',
 'realtime_train_infer_ddpg_2635',
 'remote_can_client_228',
 'remote_can_client_230',
 'remote_can_client_241',
 'remote_can_client_243',
 'remote_can_client_261',
 'remote_can_client_273',
 'remote_can_client_304',
 'remote_can_client_305',
 'remote_can_client_315',
 'remote_can_client_316',
 'remote_can_client_332',
 'remote_can_client_333'}

854

23662